In [1]:
import numpy as np
import os
import pickle
from helper_code import *
import random
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
import json
import glob




2025-08-11 14:33:10.422194: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-11 14:33:10.439571: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754890390.460679 1808799 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754890390.467538 1808799 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-11 14:33:10.490087: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))
print(os.environ.get("XLA_FLAGS"))


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]
--xla_gpu_cuda_data_dir=/home/shjee/miniconda3/envs/tf-gpu/nvvm/libdevice


In [3]:
'''random seed 설정'''
def seed_everything(seed=42):
    """
    TensorFlow 환경에서 재현 가능한 결과를 위해 모든 시드와 환경변수를 설정합니다.
    
    **사용 전 주의사항:**
      - TensorFlow 임포트 전에 아래와 같이 determinism 관련 환경변수를 설정해야 합니다.
            os.environ["TF_DETERMINISTIC_OPS"] = "1"
            os.environ["PYTHONHASHSEED"] = str(seed)
    """
    # 환경변수 재설정 (필요 시)
    os.environ["PYTHONHASHSEED"] = str(seed)
    os.environ["TF_ENABLE_ONEDNN_OPTS"] = "0"  # OneDNN 최적화 비활성화 (재현성을 위해)

    # 파이썬, numpy, TensorFlow 시드 설정
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)


# 랜덤 시드 기록 함수
def save_random_seed(seed, filepath):
    with open(filepath, 'wb') as f:
        pickle.dump(seed, f)

# 랜덤 시드 로드 함수
def load_random_seed(filepath):
    with open(filepath, 'rb') as f:
        seed = pickle.load(f)
    return seed

# 랜덤 시드 설정 및 저장
def set_random_seed(seed_save_path, use_saved_seed=False):

    # 새로운 랜덤 시드를 생성하고 저장
    random_seed = random.randint(0, 100000)
    random.seed(random_seed)
    np.random.seed(random_seed)
    tf.random.set_seed(random_seed)
    os.environ['PYTHONHASHSEED'] = str(random_seed)
    save_random_seed(random_seed, seed_save_path)
    print(f"Generated new seed: {random_seed}")


# 랜덤 시드 설정 함수 (저장된 시드 로드)
def set_random_seed_from_file(seed_save_path):
    # 저장된 시드를 로드하여 사용
    random_seed = load_random_seed(seed_save_path)
    print(f"Using saved seed: {random_seed}")
    
        # 랜덤 시드 설정
    # Python 내장 random 시드 설정
    random.seed(random_seed)
    np.random.seed(random_seed)
    tf.random.set_seed(random_seed)
    os.environ['PYTHONHASHSEED'] = str(random_seed)


    return random_seed

In [4]:
random_seed_path = r'C:\Users\medai01\Desktop\ecg\random_seed.pkl'
# set_random_seed_from_file(random_seed_path)

In [5]:
# Define the Challenge lead sets. These variables are not required. You can change or remove them.
twelve_leads = ('I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6')
six_leads = ('I', 'II', 'III', 'aVR', 'aVL', 'aVF')
four_leads = ('I', 'II', 'III', 'V2')
three_leads = ('I', 'II', 'V2')
two_leads = ('I', 'II')
lead_sets = (twelve_leads, six_leads, four_leads, three_leads, two_leads)

In [6]:
def processed_ecg_data(base_path):
    hea_file_paths = []
    mat_file_paths = []

    for subdir, dirs, _ in os.walk(base_path):
        for dir in dirs:
            g_dirs_path = os.path.join(subdir, dir, "g*")
            g_dirs = glob.glob(g_dirs_path)

            for g_dir in g_dirs:
                dir_hea_files = glob.glob(os.path.join(g_dir, "*.hea"))
                dir_mat_files = glob.glob(os.path.join(g_dir, "*.mat"))

                # .hea 파일과 .mat 파일이 모두 존재하는 경우에만 리스트에 추가
                for hea_file in dir_hea_files:
                    base_file_name = os.path.splitext(hea_file)[0]
                    mat_file = base_file_name + ".mat"
                    if mat_file in dir_mat_files:
                        hea_file_paths.append(hea_file)
                        mat_file_paths.append(mat_file)

    return hea_file_paths, mat_file_paths


model_path = '/home/shjee/projects/ecg1d/model'
base_path = '/home/shjee/projects/ecg1d/physionet.org/training'
_ ,  X_files = processed_ecg_data(base_path)


In [7]:
print(X_files[:10])

['/home/shjee/projects/ecg1d/physionet.org/training/chapman_shaoxing/g4/JS03210.mat', '/home/shjee/projects/ecg1d/physionet.org/training/chapman_shaoxing/g4/JS03713.mat', '/home/shjee/projects/ecg1d/physionet.org/training/chapman_shaoxing/g4/JS03831.mat', '/home/shjee/projects/ecg1d/physionet.org/training/chapman_shaoxing/g4/JS03583.mat', '/home/shjee/projects/ecg1d/physionet.org/training/chapman_shaoxing/g4/JS04023.mat', '/home/shjee/projects/ecg1d/physionet.org/training/chapman_shaoxing/g4/JS03735.mat', '/home/shjee/projects/ecg1d/physionet.org/training/chapman_shaoxing/g4/JS04135.mat', '/home/shjee/projects/ecg1d/physionet.org/training/chapman_shaoxing/g4/JS03621.mat', '/home/shjee/projects/ecg1d/physionet.org/training/chapman_shaoxing/g4/JS03527.mat', '/home/shjee/projects/ecg1d/physionet.org/training/chapman_shaoxing/g4/JS03202.mat']


In [8]:
def save_train_dict(train_dict, save_path):
    """
    train_dict 데이터를 파일로 저장하는 함수.

    """
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    # JSON으로 저장할 데이터
    json_data = {
        "header_files": train_dict["header_files"],
        "recording_files": train_dict["recording_files"],
        "classes": train_dict["classes"]
    }

    # JSON 저장
    json_file = os.path.join(save_path, "metadata.json")
    with open(json_file, "w") as f:
        json.dump(json_data, f, indent=4)
    
    # NPY로 저장할 데이터
    npy_keys = ["train_labels", "test_labels", "val_labels", "train_files", "test_files", "val_files"]
    
    for key in npy_keys:
        npy_file = os.path.join(save_path, f"{key}.npy")
        np.save(npy_file, train_dict[key])

    print(f"Train dictionary saved at {save_path}")

def load_train_dict(load_path):
    """
    저장된 데이터를 불러와 train_dict 형태로 변환하는 함수.
    """
    if not os.path.exists(load_path):
        raise FileNotFoundError(f"{load_path} does not exist.")

    # JSON 데이터 로드
    json_file = os.path.join(load_path, "metadata.json")
    with open(json_file, "r") as f:
        json_data = json.load(f)

    # NPY 데이터 로드
    npy_keys = ["train_labels", "test_labels", "val_labels", "train_files", "test_files", "val_files"]
    npy_data = {key: np.load(os.path.join(load_path, f"{key}.npy"), allow_pickle=True) for key in npy_keys}

    # 복원된 train_dict
    train_dict = {**json_data, **npy_data}

    print(f"Train dictionary loaded from {load_path}")
    return train_dict


# # 저장
# save_train_dict(train_dict, "train_dict_storage")

# 불러오기
# train_dict = load_train_dict("train_dict_storage")

In [9]:
batch_size = 256
epochs = 60
signal_len = 5000
model_directory = model_path

'''tensorboard --logdir=logs/fit --port=6006'''

'tensorboard --logdir=logs/fit --port=6006'

In [10]:
def check_missing_headers(X_files):
    missing_headers = []

    for mat_path in X_files:
        hea_path = os.path.splitext(mat_path)[0] + ".hea"
        if not os.path.exists(hea_path):
            print(f"[헤더 없음] {hea_path}")
            missing_headers.append(hea_path)

    print(f"\n총 {len(missing_headers)}개의 헤더가 누락되었습니다.")
    
    return missing_headers


missing = check_missing_headers(X_files)



총 0개의 헤더가 누락되었습니다.


In [11]:
from team_code import load_data, load_data_2, train_model_2

X_train_all, y_train_all, X_val_all, y_val_all, X_test_all, y_test_all, abbreviation_sorted = load_data_2(X_files)
# X_train_all, y_train_all = load_data(train_files)
# X_val_all, y_val_all  = load_data(val_files)

load_start
85376
=== Train ===
NSR: 22866
SB: 14931
TAb: 9163
STach: 7054
PVC/VPB: 6507
AFL: 6342
LAD: 6064
AF: 4142
TInv: 3075
CRBBB/RBBB: 2814
IAVB: 2793
SA: 2706
PAC/SVPB: 2554
LAnFB: 1726
QAb: 1644
LQT: 1511
IRBBB: 1474
NSIVCB: 1390
LQRSV: 1260
CLBBB/LBBB: 1167
RAD: 989
PR: 907
PRWP: 482

=== Validation ===
NSR: 2858
SB: 1867
TAb: 1145
STach: 882
PVC/VPB: 818
AFL: 792
LAD: 758
AF: 517
TInv: 384
CRBBB/RBBB: 351
IAVB: 349
SA: 338
PAC/SVPB: 319
LAnFB: 216
QAb: 205
LQT: 189
IRBBB: 184
NSIVCB: 174
LQRSV: 158
CLBBB/LBBB: 145
RAD: 124
PR: 114
PRWP: 60

=== Test ===
NSR: 2858
SB: 1866
TAb: 1146
STach: 882
PVC/VPB: 808
AFL: 793
LAD: 758
AF: 518
TInv: 384
CRBBB/RBBB: 352
IAVB: 349
SA: 338
PAC/SVPB: 319
LAnFB: 216
QAb: 205
LQT: 189
IRBBB: 184
NSIVCB: 174
LQRSV: 157
CLBBB/LBBB: 145
RAD: 124
PR: 113
PRWP: 60

[INFO] 클래스별 개수 분포를 label_distribution.xlsx로 저장했습니다.
load_end


In [12]:
with open('abbreviation_sorted.txt', 'w', encoding='utf-8') as f:
    for abbr in abbreviation_sorted:
        f.write(abbr + '\n')

In [ ]:

from tensorflow.keras.backend import clear_session
from team_code import load_data, load_data_2, train_model_2, X_rawecg, X_dwt, X_pan_tomkins, train_model_3
from tensorflow.keras import mixed_precision

with tf.device('/device:GPU:1'):
    policy = mixed_precision.Policy('mixed_float16')  # FP16 연산 적용
    mixed_precision.set_global_policy(policy)

    filters = ['Raw', 'DWT', 'PT']
    lr =  0.0001

    for i, f in enumerate(filters) :
        for leads in lead_sets:
            num_leads = len(leads)

            if (i, num_leads) not in {(1, 3)}:
                continue


            '''전처리 과정 함수'''
            if i ==  0 :
                X_train_all_filtered = X_rawecg(X_train_all, num_leads)
                X_val_all_filtered = X_rawecg(X_val_all, num_leads)
                print("filtering complete")

            if i ==  1 :
                X_train_all_filtered = X_dwt(X_train_all, num_leads)
                X_val_all_filtered = X_dwt(X_val_all, num_leads)
                print("filtering complete")

            if i ==  2 :
                X_train_all_filtered = X_pan_tomkins(X_train_all, num_leads)
                X_val_all_filtered = X_pan_tomkins(X_val_all, num_leads)
                print("filtering complete")

            model_name = f"{f}_model_{num_leads}_leads_lr{lr}"

            train_model_3(X_train_all_filtered, y_train_all, X_val_all_filtered, y_val_all, num_leads, batch_size, lr,  epochs, signal_len, model_name, model_directory, i)
            

I0000 00:00:1754890486.461658 1808799 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 43184 MB memory:  -> device: 0, name: NVIDIA L40S, pci bus id: 0000:b8:00.0, compute capability: 8.9
I0000 00:00:1754890486.462277 1808799 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 881 MB memory:  -> device: 1, name: NVIDIA L40S, pci bus id: 0000:c8:00.0, compute capability: 8.9
Preprocessing with DWT + bandpass + standardization: 100%|██████████| 61707/61707 [02:00<00:00, 511.38it/s]
Preprocessing with DWT + bandpass + standardization: 100%|██████████| 7718/7718 [00:14<00:00, 517.45it/s]


filtering complete


W0000 00:00:1754890624.905420 1810046 gpu_backend_lib.cc:579] Can't find libdevice directory ${CUDA_DIR}/nvvm/libdevice. This may result in compilation or runtime failures, if the program we try to run uses routines from libdevice.
Searched for CUDA in the following directories:
  /home/shjee/miniconda3/envs/tf-gpu/nvvm/libdevice
  ipykernel_launcher.runfiles/cuda_nvcc
  ipykern/cuda_nvcc
  
  /usr/local/cuda
  /home/shjee/miniconda3/envs/tf-gpu/lib/python3.10/site-packages/tensorflow/python/platform/../../../nvidia/cuda_nvcc
  /home/shjee/miniconda3/envs/tf-gpu/lib/python3.10/site-packages/tensorflow/python/platform/../../../../nvidia/cuda_nvcc
  /home/shjee/miniconda3/envs/tf-gpu/lib/python3.10/site-packages/tensorflow/python/platform/../../cuda
  .
You can choose the search directory by setting xla_gpu_cuda_data_dir in HloModule's DebugOptions.  For most apps, setting the environment variable XLA_FLAGS=--xla_gpu_cuda_data_dir=/path/to/cuda will work.


Train Start...

Epoch 1: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 1/60


I0000 00:00:1754890644.000834 1809626 service.cc:148] XLA service 0x7380b4003240 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1754890644.000896 1809626 service.cc:156]   StreamExecutor device (0): NVIDIA L40S, Compute Capability 8.9
I0000 00:00:1754890644.000905 1809626 service.cc:156]   StreamExecutor device (1): NVIDIA L40S, Compute Capability 8.9
2025-08-11 14:37:24.539458: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1754890649.348344 1809626 cuda_dnn.cc:529] Loaded cuDNN version 91100
2025-08-11 14:38:12.751423: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'input_convert_reduce_fusion_19', 8 bytes spill stores, 8 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'input_convert_red

...

241/241 ━━━━━━━━━━━━━━━━━━━━ 17s 69ms/step - dynamic_f1: 0.1256 - loss: 1.8021 - val_dynamic_f1: 0.1252 - val_loss: 1.7595 - learning_rate: 3.1250e-05

Epoch 57: LearningRateScheduler setting learning rate to 3.125000148429535e-05.
Epoch 57/60
241/241 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - dynamic_f1: 0.1246 - loss: 1.8036
Epoch 57: saving model to /home/shjee/projects/ecg1d/model/PT_model_3_leads_lr0.0001_e57_valf10.13.h5


241/241 ━━━━━━━━━━━━━━━━━━━━ 17s 71ms/step - dynamic_f1: 0.1246 - loss: 1.8036 - val_dynamic_f1: 0.1255 - val_loss: 1.7547 - learning_rate: 3.1250e-05

Epoch 58: LearningRateScheduler setting learning rate to 3.125000148429535e-05.
Epoch 58/60
241/241 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - dynamic_f1: 0.1252 - loss: 1.8018
Epoch 58: saving model to /home/shjee/projects/ecg1d/model/PT_model_3_leads_lr0.0001_e58_valf10.13.h5


241/241 ━━━━━━━━━━━━━━━━━━━━ 16s 68ms/step - dynamic_f1: 0.1252 - loss: 1.8018 - val_dynamic_f1: 0.1250 - val_loss: 1.7555 - learning_rate: 3.1250e-05

Epoch 59: LearningRateScheduler setting learning rate to 3.125000148429535e-05.
Epoch 59/60
241/241 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - dynamic_f1: 0.1254 - loss: 1.8015
Epoch 59: saving model to /home/shjee/projects/ecg1d/model/PT_model_3_leads_lr0.0001_e59_valf10.13.h5


241/241 ━━━━━━━━━━━━━━━━━━━━ 17s 69ms/step - dynamic_f1: 0.1254 - loss: 1.8015 - val_dynamic_f1: 0.1258 - val_loss: 1.7553 - learning_rate: 3.1250e-05

Epoch 60: LearningRateScheduler setting learning rate to 3.125000148429535e-05.
Epoch 60/60
241/241 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - dynamic_f1: 0.1242 - loss: 1.8004
Epoch 60: saving model to /home/shjee/projects/ecg1d/model/PT_model_3_leads_lr0.0001_e60_valf10.12.h5


241/241 ━━━━━━━━━━━━━━━━━━━━ 17s 70ms/step - dynamic_f1: 0.1242 - loss: 1.8004 - val_dynamic_f1: 0.1243 - val_loss: 1.7564 - learning_rate: 3.1250e-05


In [14]:
'''Test, validation 데이터 저장'''

# np.savez('ecg_test_data.npz', X=X_test_all, y=y_test_all)
# np.savez('ecg_val_data.npz', X=X_val_all, y=y_val_all)


'Test, validation 데이터 저장'